#We will collect text data using Twitter API.

In [ ]:
!pip install tweepy

In [ ]:
!pip install openpyxl

In [ ]:
import numpy as np
import pandas as pd
import tweepy
import json
from tweepy import OAuthHandler

In [ ]:
consumer_key = "zsYVyAx81uMJVdMB9KBOEndjC"
consumer_secret = "NlMYKvJoL9lbx3VdahgJNsLg35QrFXPSwDCKl7pPArbpxOEFSG"
access_token = "3052766490-qRwNpxcKK5QXu3yazwQMd4B5NzWgaVUYeO9PLXe"
access_token_secret = "wDzrraoXVUnJUXimh2q0nsFfqdiLxgAzMxH2XHacgicWq"

In [ ]:
auth = tweepy.OAuthHandler( consumer_key , consumer_secret )
auth.set_access_token( access_token , access_token_secret )
api = tweepy.API(auth)

In [ ]:
query = "RCB winning IPL"

In [ ]:
Tweets = api.search( query , count=10 , lang='en' , exclude='retweets' , tweet_mode='extended' )
# tweet_mode='extended'
# tweet_mode='compat'

for tweet in Tweets:
    print(tweet,"\n")
    print(".....................................\n")

Status(_api=<tweepy.api.API object at 0x7f6fe7a7ea20>, _json={'created_at': 'Mon Nov 02 21:21:54 +0000 2020', 'id': 1323374803684749313, 'id_str': '1323374803684749313', 'full_text': '#RCB please release #ABD, he deserves to be a part of IPL winning team. \n#RCBvsDC \n#iplseason13', 'truncated': False, 'display_text_range': [0, 95], 'entities': {'hashtags': [{'text': 'RCB', 'indices': [0, 4]}, {'text': 'ABD', 'indices': [20, 24]}, {'text': 'RCBvsDC', 'indices': [73, 81]}, {'text': 'iplseason13', 'indices': [83, 95]}], 'symbols': [], 'user_mentions': [], 'urls': []}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1313338901696704517, 'id_str': '1313338901696704517', 'name': 'Rohit yadav', 'screen_name': 'Rohitya

#Get the tweets and some Attributes

In [ ]:
df = pd.DataFrame(columns = ['Tweets' , 'User' , 'User_statuses_count' , 
                            'user_followers' , 'User_location' , 'User_verified' ,
                            'fav_count' , 'rt_count' , 'tweet_date'] )
print(df)
# print(df.shape)

Empty DataFrame
Columns: [Tweets, User, User_statuses_count, user_followers, User_location, User_verified, fav_count, rt_count, tweet_date]
Index: []


In [ ]:
def stream(data, file_name):
    i = 0
    for tweet in tweepy.Cursor(api.search, q=data, count=100, lang='en').items():
        print(i, end='\r')
        df.loc[i, 'Tweets'] = tweet.text
        df.loc[i, 'User'] = tweet.user.name
        df.loc[i, 'User_statuses_count'] = tweet.user.statuses_count
        df.loc[i, 'user_followers'] = tweet.user.followers_count
        df.loc[i, 'User_location'] = tweet.user.location
        df.loc[i, 'User_verified'] = tweet.user.verified
        df.loc[i, 'fav_count'] = tweet.favorite_count
        df.loc[i, 'rt_count'] = tweet.retweet_count
        df.loc[i, 'tweet_date'] = tweet.created_at
        df.to_excel('{}.xlsx'.format(file_name))
        i = i+1
        if i == 1000:
            break
        else:
            pass

In [ ]:
stream(data=["RCB winning IPL"] , file_name='my_tweets')

In [ ]:
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date
0,@Underta54955802 @AbdPakku @Viratkohli18f_c @I...,ssngage,177,0,,False,1,0,2020-11-02 20:29:44
1,I will be really surprised to see @mipaltan\n ...,Sreek_kiran_CRIC,544,78,"Dresden, Germany",False,3,0,2020-11-02 19:52:00
2,Anyways defeating MI twice in playoffs to win ...,Adam Griffith⚪(Gendbaazi Shikshak),1037,66,Chinnaswamy Stadium,False,5,0,2020-11-02 19:04:03
3,@Shreya_V21 @DebilitatinShit And MI is winning...,Maybe,868,114,,False,0,0,2020-11-02 19:03:57
4,#IPL2020 #SRHvMI | MI Predicted XI against SRH...,HT Sports,111487,101406,India,True,2,0,2020-11-02 18:54:37


In [ ]:
!pip install textblob

Sentiment(polarity, subjectivity)

Polarity score: [-1 , +1]

Subjectivity: [0.0 , 1.0] 
where 0.0 is very Objective and 1.0 is very Subjective.

In [21]:
from textblob import TextBlob

In [22]:
import re
def clean_tweet(tweet):
    return ' '.join(re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)', ' ', tweet).split())

In [23]:
def analyze_sentiment(tweet):
    analysis = TextBlob(tweet)
    if analysis.sentiment.polarity > 0:
        return 'Positive'
    elif analysis.sentiment.polarity == 0:
        return 'Neutral'
    else:
        return 'Negative'

In [ ]:
df['clean_tweet'] = df['Tweets'].apply(lambda x : clean_tweet(x))

In [25]:
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date,clean_tweet
0,"#RCB please release #ABD, he deserves to be a ...",Rohit yadav,57,4,,False,0,0,2020-11-02 21:21:54,RCB please release ABD he deserves to be a par...
1,@Underta54955802 @AbdPakku @Viratkohli18f_c @I...,ssngage,177,0,,False,1,0,2020-11-02 20:29:44,c Really That s why his winning is more than a...
2,@ThorOffl @stranger2898 Devdutta\nMoeen\nVirat...,🆆🆁🅾🅽🅶 🆃🆄🆁🅽༒,14477,178,,False,2,0,2020-11-02 20:13:22,Devdutta Moeen Virat Finch Abd Dube Morris Sun...
3,@imVkohli @RCBTweets We r so happy that u qual...,Shamiya Shami Iraqui,194,27,India,False,0,0,2020-11-02 19:08:40,We r so happy that u qualified but we didn t e...
4,I don't have much to live. My last wish is to ...,Mufaddal Johns,48,3,,False,0,0,2020-11-02 19:01:40,I don t have much to live My last wish is to s...


In [26]:
df['Sentiment'] = df['clean_tweet'].apply(lambda x : analyze_sentiment(x) )
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date,clean_tweet,Sentiment
0,"#RCB please release #ABD, he deserves to be a ...",Rohit yadav,57,4,,False,0,0,2020-11-02 21:21:54,RCB please release ABD he deserves to be a par...,Positive
1,@Underta54955802 @AbdPakku @Viratkohli18f_c @I...,ssngage,177,0,,False,1,0,2020-11-02 20:29:44,c Really That s why his winning is more than a...,Positive
2,@ThorOffl @stranger2898 Devdutta\nMoeen\nVirat...,🆆🆁🅾🅽🅶 🆃🆄🆁🅽༒,14477,178,,False,2,0,2020-11-02 20:13:22,Devdutta Moeen Virat Finch Abd Dube Morris Sun...,Positive
3,@imVkohli @RCBTweets We r so happy that u qual...,Shamiya Shami Iraqui,194,27,India,False,0,0,2020-11-02 19:08:40,We r so happy that u qualified but we didn t e...,Positive
4,I don't have much to live. My last wish is to ...,Mufaddal Johns,48,3,,False,0,0,2020-11-02 19:01:40,I don t have much to live My last wish is to s...,Positive


In [27]:
n = 1
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 @Underta54955802 @AbdPakku @Viratkohli18f_c @IPL Really? That's why his winning % is more than any other Indian cap… https://t.co/vu4Vj01dyZ

Clean tweet:
 c Really That s why his winning is more than any other Indian cap

Sentiment of the tweet:
 Positive


In [28]:
n = 20
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 I was supporting RCB to win this match. And I also knew RCB is gonna win this year's IPL trophy.

But this "win" wi… https://t.co/kmmnZ72DU2

Clean tweet:
 I was supporting RCB to win this match And I also knew RCB is gonna win this year s IPL trophy But this win wi

Sentiment of the tweet:
 Positive


In [29]:
n = 203
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 RT @desi_bhayo88: #SRHvsRCB 
No one

Le RCB fans every year after losing the hope of winning the ipl https://t.co/OfmFHgvysI

Clean tweet:
 RT bhayo88 SRHvsRCB No one Le RCB fans every year after losing the hope of winning the ipl

Sentiment of the tweet:
 Positive


In [30]:
df[df.Sentiment == 'Positive'].shape[0]

945

In [31]:
df[df.Sentiment == 'Neutral'].shape[0]

22

In [32]:
df[df.Sentiment == 'Negative'].shape[0]

33